In [1]:
import pandas as pd
import numpy as np
import json
import os
from tqdm import tqdm

## Load journal and conferences lists

Set of journal and conferences from [gov.pl](https://www.gov.pl/web/edukacja-i-nauka/nowy-rozszerzony-wykaz-czasopism-naukowych-i-recenzowanych-materialow-z-konferencji-miedzynarodowych) with impact metrics

In [2]:
# load https://www.gov.pl/web/edukacja-i-nauka/nowy-rozszerzony-wykaz-czasopism-naukowych-i-recenzowanych-materialow-z-konferencji-miedzynarodowych
# sheet 1: 'journals'
gov_articles_data = pd.read_excel("..//data//external//Wykaz_dyscyplin_do_czasopism_i_materiałów_konferencyjnych.xlsx", 0, header=None)

gov_articles_data.columns = np.where(gov_articles_data.iloc[0].notna(), 
                                     gov_articles_data.iloc[0].astype(str) + ' - ' + gov_articles_data.iloc[1].astype(str), 
                                     gov_articles_data.iloc[1])
gov_articles_data = gov_articles_data[2:]
gov_articles_data = gov_articles_data.reset_index(drop=True)

gov_articles_data = gov_articles_data.drop('Lp.', axis=1)

display(gov_articles_data)

,Unikatowy Identyfikator Czasopisma,Tytuł 1,issn,e-issn,Tytuł 2,issn,e-issn,Punkty,archeologia - 101,filozofia - 102,...,prawo kanoniczne - 510,psychologia - 511,astronomia - 601,informatyka - 602,matematyka - 603,nauki biologiczne - 604,nauki chemiczne - 605,nauki fizyczne - 606,nauki o Ziemi i środowisku - 607,nauki teologiczne - 701
0,1,2D Materials,2053-1583,2053-1583,2D Materials,NaN,2053-1583,140,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,x,x,NaN,NaN
1,2,3 Biotech,2190-572X,2190-5738,3 Biotech,2190-572X,2190-5738,70,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN
2,3,3C Empresa,2254-3376,2254-3376,NaN,NaN,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,3c Tecnologia,2254-4143,2254-4143,NaN,NaN,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,3C Tic,2254-6529,2254-6529,NaN,NaN,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31428,499029,Open Political Science,,2543-8042,NaN,NaN,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31429,499067,Językoznawstwo,1897-0389,2391-5137,Linguistics,NaN,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31430,499108,Człowiek i Społeczeństwo,0239-3271,,Man and Society,NaN,NaN,40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31431,499116,Com.press,,2545-2320,NaN,NaN,NaN,20,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Get journal titles list

In [3]:
journals_list = gov_articles_data[['Tytuł 2', 'Punkty']]
journals_list = journals_list.drop_duplicates(subset=['Tytuł 2'])
journals_list = journals_list.dropna()
journals_list = journals_list.set_index('Tytuł 2')
display(journals_list)

,Punkty
Tytuł 2,
2D Materials,140
3 Biotech,70
3D Printing and Additive Manufacturing,100
3D Research,40
"3L: Language, Linguistics, Literature",70
...,...
The Old and the New Ages,20
Language Education,20
Tax advisory ‐ Tax Studies Institute Bulletin,40


In [4]:
# load https://www.gov.pl/web/edukacja-i-nauka/nowy-rozszerzony-wykaz-czasopism-naukowych-i-recenzowanych-materialow-z-konferencji-miedzynarodowych
# sheet 1: 'conferences'
gov_conferences_data = pd.read_excel("..//data//external//Wykaz_dyscyplin_do_czasopism_i_materiałów_konferencyjnych.xlsx", 1)
gov_conferences_data = gov_conferences_data.drop('LP.', axis=1)

gov_conferences_data['Przypisane dyscypliny naukowe'] = gov_conferences_data['Przypisane dyscypliny naukowe'].replace('\n', ' ', regex=True)

display(gov_conferences_data)

,Nazwa konferencji,Liczba punktów,Przypisane dyscypliny naukowe
0,3-D Digital Imaging and Modelling [3DIM],20,informatyka techniczna i telekomunikacja; info...
1,A Satellite workshop on Formal Approaches to T...,20,informatyka techniczna i telekomunikacja; info...
2,Accounting and Finance Association of Australi...,20,informatyka techniczna i telekomunikacja; info...
3,ACIS Conference on Software Engineering Resear...,20,informatyka techniczna i telekomunikacja; info...
4,ACM Annual Computer Science Conference [CSC],20,informatyka techniczna i telekomunikacja; info...
...,...,...,...
1633,International Conference: Sciences of Electron...,20,informatyka techniczna i telekomunikacja; info...
1634,The Symposium of Combinatorial Search [SoCS],20,informatyka techniczna i telekomunikacja; info...
1635,The International Conference on Verification a...,20,informatyka techniczna i telekomunikacja; info...
1636,International Teletraffic Congress [ITC],20,informatyka techniczna i telekomunikacja; info...


Get conferences titles list


In [5]:
conferences_list = gov_conferences_data[['Nazwa konferencji', 'Liczba punktów']]
conferences_list = conferences_list.drop_duplicates(subset=['Nazwa konferencji'])
conferences_list = conferences_list.dropna()
conferences_list = conferences_list.set_index('Nazwa konferencji')
display(conferences_list)

,Liczba punktów
Nazwa konferencji,
3-D Digital Imaging and Modelling [3DIM],20
A Satellite workshop on Formal Approaches to Testing of Software [FATES],20
Accounting and Finance Association of Australia and New Zealand Conference [AFAANZ],20
"ACIS Conference on Software Engineering Research, Management and Applications [SERA]",20
ACM Annual Computer Science Conference [CSC],20
...,...
"International Conference: Sciences of Electronics, Technologies of information and Telecommunication [SETIT]",20
The Symposium of Combinatorial Search [SoCS],20
The International Conference on Verification and Evaluation of Computer and Communication Systems [VECoS],20


## Load collection of computer science publications

A collection of data on publications in the field of computer science from [dblp](https://paperswithcode.com/dataset/dblp)

In [6]:
# load https://www.aminer.org/citation v10
directory_path = "../data/external/dblp-ref-10"  # Zastąp to ścieżką do katalogu, który chcesz przeszukać

publications = []

try:
    file_names = os.listdir(directory_path)

    # Wyświetl nazwy plików
    for file_name in file_names:
        print(file_name)
        with open(os.path.join(directory_path, file_name)) as input_file:
            for line in tqdm(input_file):
                publication = json.loads(line)
                if 'references' not in publication:
                    publication['references'] = []
                if 'authors' not in publication:
                    publication['authors'] = []
                publications.append(publication)
        
except FileNotFoundError:
    print(f"Katalog '{directory_path}' nie istnieje.")

except PermissionError:
    print(f"Brak uprawnień do odczytu katalogu '{directory_path}'.")

display(len(publications))

dblp-ref-0.json


1000000it [00:10, 91117.59it/s]


dblp-ref-1.json


1000000it [00:12, 81482.17it/s]


dblp-ref-2.json


1000000it [00:13, 72404.71it/s]


dblp-ref-3.json


79007it [00:00, 94102.22it/s]


3079007

## Concatenate datasets

First, convert the list to a dictionary; this allows us to search by title using a hash with **O(1)** complexity.

In [7]:
# convert articles list
journal_points_lookup = journals_list.to_dict(orient='index')

# convert conferences list
conference_points_lookup = conferences_list.to_dict(orient='index')

Next find matching venues & titles

In [8]:
processed_publications = []

for publication in tqdm(publications):
    if publication['venue'] not in conference_points_lookup and publication['venue'] not in journal_points_lookup:
        continue

    processed_publication = {key: publication[key] for key in ["id", "title", "year", "references", "authors", "n_citation", "venue"]}
    if publication['venue'] in conference_points_lookup:
        processed_publication['gov_score'] = conference_points_lookup[publication['venue']]['Liczba punktów']
    elif publication['venue'] in journal_points_lookup:
        processed_publication['gov_score'] = journal_points_lookup[publication['venue']]['Punkty']
    else:
        continue

    processed_publications.append(processed_publication)

del publications

print(len(processed_publications))

100%|████████████████████████████████████████████████████████████████████| 3079007/3079007 [00:06<00:00, 472320.37it/s]


850406


**Note**
When we filter by *venue* in both list program match the vast majority of records than when we use *title* for publications and *venue* for conferences separately (about 18 rows)

## Save filtered data

In [9]:
tqdm.pandas()

processed_publications = pd.DataFrame(processed_publications)
processed_publications.progress_apply(lambda row: row, axis=1).to_csv(os.path.join("../data/interim", "articles_with_score_df.csv"), index=False)

100%|███████████████████████████████████████████████████████████████████████| 850406/850406 [00:22<00:00, 38408.70it/s]


## Check results

In [10]:
check_df = pd.read_csv("../data/interim//articles_with_score_df.csv")

display(check_df)

,id,title,year,references,authors,n_citation,venue,gov_score
0,00dc2bba-3237-4d4e-b541-1205b97df981,Software Evolution through Transformations.,2003,[],"['Reiko Heckel', 'Tom Mens', 'Michel Wermeling...",50,Electronic Notes in Theoretical Computer Science,40
1,019a51de-5136-41cb-81fa-ed659c915913,Context Dependent Automatic Textile Image Anno...,2007,[],"['Yosuke Furukawa', 'Yusuke Kamoi', 'Tatsuya S...",50,Journal of Advanced Computational Intelligence...,20
2,03385413-7c3a-44ce-b40c-58ff21304270,On the signed total chromatic number of a graph.,2006,[],['Michael A. Henning'],7,Ars Combinatoria,40
3,034261a4-1e9b-463f-9549-c0b890af6a41,Traveling wave solutions of the n-dimensional ...,2012,"['36905344-ad94-4e1c-8c44-d535e5d53fcf', 'ac39...",['Robert A. Van Gorder'],6,Applied Mathematics Letters,100
4,092bdf49-c5fc-4183-a027-b54c4addcdae,Special issue on causal modeling: Preface.,1989,[],['Werner Horn'],0,Applied Artificial Intelligence,40
...,...,...,...,...,...,...,...,...
850401,fe9de131-7dd5-4345-90f8-f35bcd5e795f,Condorcet Consistent Bundling with Social Choice.,2017,[],"['Shreyas Sekar', 'Sujoy Sikdar', 'Lirong Xia']",0,Autonomous Agents and Multi-Agent Systems,70
850402,febf1d16-9501-470b-9a12-271aa60f8bad,Cloudy with a Chance of Poaching: Adversary Be...,2017,[],"['Debarun Kar', 'Benjamin J. Ford', 'Shahrzad ...",0,Autonomous Agents and Multi-Agent Systems,70
850403,fee4a0fb-3dd4-4d0c-9a5e-f7def73f5a6e,A Deterministic Distributed Algorithm for Reas...,2017,[],"['Shufeng Kong', 'Jae Hee Lee', 'Sanjiang Li']",50,Autonomous Agents and Multi-Agent Systems,70
850404,ff5ce050-ea8d-40e8-a25f-c629bed2ff9c,Mobile Platforms and Multi-Mobile Platform Dev...,2014,[],"['Hassan Charaf', 'Peter Ekler', 'Tamás Mészár...",50,Acta Cybernetica,20
